# 1. Setup and Get Data

### 1.1 Install Dependencies and Setup

In [27]:
%pip install --user labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


### 1.2 Collect Images using OpenCV

In [28]:
import os
import time
import uuid
import cv2

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


### 1.3 Annotate Images

In [ ]:
!labelme